### **Aplicaciones**


#### Caché

Con caché nos referimos al proceso de almacenar cierta información obtenida en un sistema de almacenamiento rápido A, para tenerla lista en caso de que necesitemos leerla nuevamente en un futuro cercano. Los datos pueden haber sido previamente recuperados de un sistema de almacenamiento más lento B o ser el resultado de un cómputo intensivo en CPU.

Para las aplicaciones web, la escalabilidad es una preocupación importante, probablemente el aspecto más debatido en las revisiones de diseño para cualquier producto que aspire a volverse viral.

La caché es a menudo lo único que salva a las bases de datos de literalmente incendiarse o al menos de colapsar. Pero incluso tu laptop tiene varios niveles de caché, desde la caché L1 rápida dentro de su CPU hasta la caché en memoria que se usa para procesar archivos grandes. Tu propio sistema operativo almacena en caché páginas de memoria dentro y fuera de la RAM, intercambiándolas al disco, para tener un espacio de direcciones virtual mayor y darte la impresión de que tiene más memoria disponible de la que realmente está instalada en tu máquina.

En otras palabras, las cachés son uno de los fundamentos de los sistemas IT modernos. Por supuesto, dado que el almacenamiento rápido es más costoso, solo hay una cantidad limitada de él, y la mayoría de los datos tendrá que quedar fuera de la caché.

El algoritmo que se utiliza para decidir qué datos permanecen en la caché determina el comportamiento de la caché y la tasa de aciertos (cuando los datos buscados ya están en la caché) y fallos. Algunos de los algoritmos más usados son LRU (Least Recently Used), MRU (Most Recently Used) y LFU (Least Frequently Used). En otras palabras, tienen dificultades con objetos, ubicaciones de memoria o páginas web solicitadas solo una vez, y nunca más (en la vida promedio de la caché). Esto es particularmente común en routers y redes de distribución de contenido (CDN), donde un promedio del 75% de las solicitudes para un nodo son one-hit wonders.

Utilizar un diccionario para llevar un registro de las solicitudes nos permite almacenar un objeto en la caché solo cuando se solicita por segunda vez, filtrando los one-hit wonders y mejorando la tasa de aciertos de la caché. Los Bloom filters permiten realizar estas búsquedas utilizando operaciones de tiempo constante amortizado y con espacio limitado, a costa de aceptar algunos falsos positivos sin consecuencias. Para esta aplicación, el único resultado de los falsos positivos, sin embargo, sería una pequeña reducción en la ganancia de rendimiento de la caché que obtenemos al usar el Bloom filter en primer lugar (así que, ningún daño hecho).

#### Routers

Los routers modernos tienen espacio limitado y, con el volumen de paquetes que procesan por segundo, necesitan algoritmos extremadamente rápidos. Son, por lo tanto, el destinatario perfecto para los Bloom filters, para todas aquellas operaciones que pueden tolerar una pequeña tasa de errores.

Además de la caché, los routers a menudo emplean Bloom filters para llevar un registro de IPs prohibidas y para mantener estadísticas que se utilizarán para detectar ataques de DoS (denegación de servicio).

#### Crawler

Los crawlers son agentes de software automatizados que escanean una red (o incluso la web entera) en busca de contenido, parseando e indexando todo lo que encuentran.

Cuando un crawler encuentra enlaces en una página o documento, usualmente está programado para seguirlos y rastrear recursivamente el destino del enlace. Hay algunas excepciones: por ejemplo, la mayoría de los tipos de archivo serán ignorados por los crawlers, al igual que los enlaces creados usando etiquetas `<a>` con un atributo `rel="nofollow"`.

Es recomendable que marques de esta manera cualquier ancla con un enlace a una acción que tenga efectos secundarios. De lo contrario, los crawlers de los motores de búsqueda, incluso si respetan esta política, causarán comportamientos impredecibles.

Lo que puede suceder es que, si escribes tu propio crawler y no tienes cuidado, podría quedar atrapado en un bucle infinito entre dos o más páginas con enlaces mutuos (o cadena de enlaces) entre sí.

Para evitar tales bucles, los crawlers necesitan llevar un registro de las páginas que ya visitaron. Los Bloom filters, nuevamente, son la mejor manera de hacerlo, porque pueden almacenar URLs de forma compacta y realizar la comprobación y el almacenamiento de las URLs en tiempo constante.

El precio que pagas aquí por los falsos positivos es un poco más alto que en los ejemplos anteriores, porque el resultado inmediato será que el crawler nunca visitará una URL que causó un falso positivo.

Para superar este problema, es posible mantener una lista completa de las URLs visitadas en un diccionario adecuado (u otro tipo de colección) almacenado en disco, y si y solo si el Bloom filter devuelve verdadero, verificar doblemente la respuesta en el diccionario. Este enfoque no permite ahorrar espacio, pero ofrece algunos ahorros en el tiempo de ejecución si hay un alto porcentaje de one-hit wonders entre las URLs.

#### IO fetcher

Otra área en la que la caché basada en Bloom filters ayuda es en la reducción de la recuperación/almacenamiento innecesario de recursos de IO costosos. El mecanismo es el mismo que con el crawling: la operación solo se realiza cuando tenemos un "fallo", mientras que los "aciertos" usualmente desencadenan una comparación más profunda (por ejemplo, en un acierto, recuperando del disco solo las primeras líneas o el primer bloque de un documento, y comparándolos).

#### Corrector ortográfico

Versiones más simples de correctores ortográficos solían emplear Bloom filters como diccionarios. Para cada palabra del texto examinado, una búsqueda en un Bloom filter validaría la palabra como correcta o la marcaría como error ortográfico. Por supuesto, la ocurrencia de falsos positivos causaría que algunos errores ortográficos pasaran desapercibidos, pero la probabilidad de que esto suceda podía ser controlada de antemano. Hoy en día, sin embargo, los correctores ortográficos aprovechan mayormente los **tries**: estas estructuras de datos ofrecen buen rendimiento en búsquedas de texto sin los falsos positivos.

#### Bases de datos y sistemas de archivos distribuidos

Cassandra utiliza Bloom filters para escaneos de índices y determinar si un SSTable tiene datos para una fila en particular.

Del mismo modo, Apache HBase utiliza Bloom filters como un mecanismo eficiente para comprobar si un StoreFile contiene una fila específica o una celda de fila-columna. Esto, a su vez, mejora la velocidad de lectura general, al filtrar las lecturas de disco innecesarias de bloques HFile que no contienen una fila o fila-columna en particular.




### **Ejercicios**

#### **Caché**

1. **Análisis de patrón de acceso**  
   - ¿Por qué los algoritmos LRU, MRU y LFU suelen fallar ante los "one-hit wonders"?  
   - Diseña un flujo de trabajo en el que, usando un Bloom filter, solo se añadan al caché las claves solicitadas por segunda vez. ¿Cómo afecta esto la tasa de aciertos y fallos en un entorno web con alto porcentaje de one-hit wonders (≈75 %)?

2. **Impacto de falsos positivos**  
   - Supón un Bloom filter con una tasa de falsos positivos del 1 %: si un elemento "no caliente" produce un falso positivo, ¿qué efecto tiene esto sobre el rendimiento global de la caché?  
   - ¿En qué situaciones un aumento moderado de falsos positivos (por ejemplo, hasta 5 %) podría seguir siendo aceptable para la aplicación?

3. **Dimensionalidad del Bloom filter**  
   - Dado un volumen esperado de solicitudes y un límite de memoria para el bit-array, ¿cómo ajustarías los parámetros `(m, k)` para lograr un buen compromiso entre tasa de falsos positivos y espacio consumido?  
   - Describe un método para reconfigurar dinámicamente el Bloom filter en tiempo de ejecución si el patrón de solicitudes crece más allá de lo previsto.

#### **Routers**

1. **Filtrado de IPs prohibidas**  
   - Describe un esquema que combine un Bloom filter en memoria con una lista persistente en disco para gestionar IPs bloqueadas: ¿cómo garantizarías que una vez bloqueada una IP no se escape por un falso negativo?  
   - ¿Qué estrategia adoptarías para refrescar o reconstruir el Bloom filter periódicamente cuando se añadan o eliminen muchas IPs de la lista negra?

2. **Detección de DoS**  
   - Plantea un diseño de métricas basadas en Bloom filters para rastrear IPs que envían un número excesivo de paquetes en un corto intervalo. ¿Cómo calibrarías el filtro para detectar ataques sin inundarlo de falsos positivos?

3. **Limitaciones de espacio y velocidad**  
   - En un router con capacidad limitada, ¿cómo balancearías la precisión del Bloom filter con la necesidad de procesar millones de paquetes por segundo?  
   - ¿Qué alternativas o extensiones al Bloom filter (por ejemplo, Counting Bloom filter) considerarías si necesitaras soportar eliminaciones de IPs?

#### **Crawler**

1. **Prevención de bucles infinitos**  
   - Explica cómo un Bloom filter ayuda a evitar visitar URL duplicadas en un crawler, y analiza el impacto de un falso positivo en el recorrido completo de un sitio.  
   - Propón un protocolo de doble verificación (Bloom filter + diccionario en disco) y evalúa su coste en tiempo y espacio según el porcentaje de URLs nuevas vs. repetidas.

2. **Política `nofollow` y efectividad**  
   - ¿Cómo integrarías la política `rel="nofollow"` al algoritmo de seguimiento de URLs con Bloom filter para optimizar el uso de recursos?  
   - Diseña un experimento comparativo que mida el ahorro de ancho de banda y CPU al ignorar enlaces marcados vs. no ignorarlos.

3. **Escalabilidad**  
   - Si tu crawler debe cubrir miles de dominios distintos, ¿cómo organizarías varios Bloom filters (por ejemplo, uno por dominio) para mantener el rendimiento?  
   - Plantea un mecanismo de distribución de filtros en clústeres de crawling para asegurar que no haya solapamientos innecesarios de URLs entre instancias.

#### **IO fetcher**

1. **Reducción de accesos a disco**  
   - Describe un sistema de IO fetcher que utilice un Bloom filter para decidir si vale la pena leer un bloque del disco. ¿Cómo medirías el beneficio real en latencia y throughput?  
   - ¿En qué casos podría el coste de un falso positivo (lectura innecesaria de disco) exceder la ganancia de evitar un fallo de caché?

2. **Nivel de granularidad**  
   - ¿Sería más eficiente aplicar el Bloom filter sobre rutas de archivo completas, sobre bloques de datos o sobre identificadores de recurso más finos (por línea)? Justifica tu elección según patrones de acceso típicos.

3. **Adaptación a patrones de IO**  
   - Propón un esquema para adaptar los parámetros del Bloom filter en función de la carga de trabajo (transaccional vs. analítica). ¿Cómo detectarías el cambio de patrón en tiempo real?


#### **Corrector ortográfico**

1. **Tasa de acierto vs. coste de falso positivo**  
   - En un Bloom filter usado como diccionario, un falso positivo deja pasar una palabra incorrecta. ¿Cómo cuantificarías el impacto de estos errores en la experiencia de usuario?  
   - ¿Cómo combinarías el Bloom filter con un segundo nivel de verificación (por ejemplo, un trie) para reducir esos falsos positivos sin degradar el rendimiento?

2. **Comparativa con tries**  
   - ¿Cuáles son las ventajas y desventajas de un Bloom filter frente a un trie para la detección de palabras en un corrector ortográfico?  
   - Diseña un experimento que compare tiempo de búsqueda, memoria empleada y tasa de error para ambos enfoques con un vocabulario de 100 000 palabras.


#### **Bases de datos y sistemas de archivos distribuidos**

1. **Cassandra y SSTables**  
   - Explica cómo un Bloom filter acelera las consultas de lectura en Cassandra. ¿Qué ocurre cuando el filtro reporta "no existe"? ¿Y cuando reporta "posiblemente existe"?  
   - ¿Cómo manejarías la recreación o actualización del filtro cuando se crean nuevos SSTables o se compactan existentes?

2. **HBase y HFiles**  
   - Describe el flujo de ejecución de una consulta de fila en HBase que emplea Bloom filters para evitar lecturas innecesarias de bloques HFile.  
   - ¿Qué parámetros de Bloom filter ajustarías en un clúster con alta variabilidad de tamaños de fila y por qué?

3. **Escalabilidad y consistencia**  
   - En un sistema distribuido, ¿cómo aseguras que cada nodo mantenga Bloom filters coherentes con sus particiones de datos?  
   - Plantea un mecanismo para propagar cambios (por ejemplo, eliminación de datos viejos) a los Bloom filters sin detener el servicio.


In [ ]:
## Tus respuestas

#### **Por qué funcionan los Bloom filters**

Es el momento de examinar más de cerca y explicar por qué un Bloom filter realmente funciona. 

Como ya se mencionó, los Bloom filters son un compromiso entre memoria y precisión. Si vas a crear una instancia de un Bloom filter con una capacidad de almacenamiento de 8 bits y luego intentas almacenar 1 millón de objetos en él, lo más probable es que no obtengas un buen rendimiento. Por el contrario, considerando un Bloom filter con un búfer de 8 bits, todo su búfer se establecería en 1 después de aproximadamente 10-20 hashes. En ese punto, todas las llamadas a `contains` simplemente devolverán verdadero, y no podrás entender si un objeto fue realmente almacenado en el contenedor.

Si en cambio, asignamos suficiente espacio y elegimos bien nuestras funciones hash, entonces los índices generados para cada clave no entrarán en conflicto, y para dos claves diferentes la superposición entre las listas de índices generados será mínima, si la hay.

Pero, ¿cuánto espacio es suficiente? Internamente, un Bloom filter traduce cada clave en una secuencia de `k` índices elegidos de entre `m` alternativas posibles: el truco es que almacenamos las claves de forma eficiente cambiando estos `k` bits en el búfer del filtro, un arreglo de bits.

Si has repasado tu álgebra escolar recientemente, quizás habrás adivinado que solo podemos representar $m^k$ diferentes secuencias de `k` elementos extraídos de m valores, sin embargo, debemos hacer dos consideraciones:

- Tenemos, en realidad, que no podemos usar todas estas secuencias. Nos gustaría que todos los índices asociados a una clave fueran diferentes (de lo contrario, en realidad almacenaríamos menos de `k` bits por clave), por lo que buscamos que todas las listas de `k` índices estén libres de duplicados.

- No estamos realmente interesados en el orden de estos `k` índices. Es completamente irrelevante si primero escribimos el bit en el índice 0 y luego el bit en el índice 3, o viceversa. Por lo tanto, podemos considerar conjuntos en lugar de secuencias.

Considerando todo, solo permitimos (al menos en teoría) una fracción de todos los posibles conjuntos de exactamente `k` índices (distintos) extraídos del rango $0\dots m-1$, y el número de todos estos posibles conjuntos (válidos) viene dado por:

$$
    \binom{m}{k} = \frac{m!}{k!(m-k)!}
$$

El coeficiente binomial en esta fórmula expresa el número de formas en que podemos extraer `k` elementos (únicos) de un conjunto de tamaño `m` y por lo tanto nos dice cuántos conjuntos con exactamente `k` índices distintos pueden devolver nuestras `k` funciones hash.

Si queremos que cada clave se mapee a un conjunto diferente de `k` índices, entonces, dados `k` y `n`, el número de claves a almacenar, podemos calcular una cota inferior para `m` el tamaño del búfer, utilizando la fórmula anterior.

Otra forma de ver la cuestión es que, dada una secuencia de m bits (el búfer), solo podemos representar $2^m$ valores diferentes, por lo que, en un momento dado, el Bloom filter solo puede estar en uno de los $2^m$ estados posibles; sin embargo, esto solo nos da una cota laxa (aunque más fácil de calcular) sobre `n`, ya que no tomaríamos en consideración que almacenaríamos `k` bits por clave ($2^m$ se convierte en una cota exacta solo para $k==1$).


#### **Por qué no hay falsos negativos**

En la versión más simple de los Bloom filters, no se permite la eliminación. Esto significa que si un bit se cambia cuando se almacena una clave, nunca volverá a establecerse en 0. Al mismo tiempo, la salida de las funciones hash es determinista y constante en el tiempo.

> Recuerda prestar atención y cumplir estas propiedades en tu implementación si necesitas serializar Bloom filters y deserializarlos posteriormente.

Así, si una búsqueda descubre que uno de los bits asociados a una clave X está en 0, entonces sabemos con certeza que X nunca fue añadida al filtro; de lo contrario, todos los bits a los que se le hace hash a X serían 1.

#### **Pero hay falsos positivos**

Lo contrario, sin embargo, ¡no se sostiene! Un ejemplo te ayudará a entender por qué. Supongamos que tenemos un Bloom filter simple con 4 bits y 2 funciones hash. Inicialmente, el búfer está vacío:

```
B = [0, 0, 0, 0]
```

Primero, insertamos el valor 1 en el Bloom filter. Supongamos que elegimos nuestras funciones hash de manera que $h_0(1) = 0$ y $h_1(1) = 2$, por lo que la clave 1 se mapea a los índices `{0,2}`. Después de actualizarlo, nuestro búfer se ve ahora así:

```
B = [1, 0, 1, 0]
```
Ahora insertamos el valor 2, y resulta que $h_0(2) = 1$ y $h_1(2) = 2$. Por lo tanto, ahora tenemos:

```
B = [1, 1, 1, 0]
```

Finalmente, supongamos que $h_0(3) = 1$ y $h_1(3) = 0$. Si realizamos una búsqueda para el valor 3 después de esas dos inserciones, ambos bits en los índices 1 y 0 habrán sido establecidos en 1, ¡incluso si nunca almacenamos 3 en nuestra instancia del filtro! Por lo tanto, 3 nos daría un falso positivo.

Por otro lado, si tuviéramos un mapeo diferente de las funciones hash, por ejemplo, $h_0(3) = 3$ y $h_1(3) = 0$, ya que el cuarto bit aún no se había establecido ($B[0]==0$), una búsqueda habría devuelto falso.

Por supuesto, este es un ejemplo simplista intencionadamente elaborado para demostrar un punto: los falsos positivos son posibles, y son más propensos a ocurrir si no se eligen los parámetros del Bloom filter cuidadosamente. Aprenderemos cómo ajustar estos parámetros, en función del número de elementos que anticipamos almacenar y de la precisión que necesitamos.



#### **Ejercicios teóricos**

1. **Clasificación de algoritmos aleatorizados**  
   - Explica por qué el método `contains` de un Bloom filter se clasifica como un algoritmo Monte Carlo y no como Las Vegas.  
   - Argumenta qué propiedades de un Bloom filter garantizan la ausencia de falsos negativos y qué condiciones permiten falsos positivos.

2. **Análisis de probabilidad de error**  
   - Dado un Bloom filter con parámetros $m$ (bits) y $k$ (funciones hash) tras $n$ inserciones, deriva la fórmula aproximada de la probabilidad de falsos positivos.  
   - Demuestra que optimizando $k$ para minimizar esa probabilidad se obtiene $k^* = \frac{m}{n}\ln 2$.

3. **Compromiso memoria–precisión**  
   - Discute cómo varía la tasa de falsos positivos al reducir la memoria disponible en un factor de 2, manteniendo constante el número de elementos esperados.  
   - ¿Por qué el Bloom filter se considera un "compromiso" entre uso de espacio y exactitud, comparado con un conjunto hash determinista?

4. **Extensiones aleatorizadas**  
   - Compara el comportamiento de un Counting Bloom filter con el del Bloom filter estándar en términos de falsos positivos y memoria adicional.  
   - Analiza cómo afectan las colisiones de hashes a la distribución de errores en un Bloomier filter, otra estructura Monte Carlo.

5. **Simulación de falsos positivos**  
   - Implementa (o utiliza) un pequeño experimento donde, para un Bloom filter con $m=1000$, $k=4$ y $n=200$, midas empíricamente la tasa de falsos positivos probando 10 000 claves no insertadas. Compara el resultado con la tasa teórica.

6. **Ajuste de parámetros**  
   - Partiendo de un patrón de inserciones que crece en el tiempo, diseña un procedimiento para recalcular dinámicamente $k$ cuando el número de elementos insertados $n$ supere un umbral, de modo que la tasa de falsos positivos no exceda un 1 %.

7. **Impacto de la variabilidad de hash**  
   - Usando distintas familias de funciones hash (por ejemplo, MD5 truncado vs. MurmurHash), compara cómo cambia la tasa de falsos positivos en un Bloom filter con parámetros fijos. ¿Qué conclusiones sacas sobre la independencia de hashes?

8. **Detección de sesgos**  
   - Genera un conjunto de entradas con distribución no uniforme (por ejemplo, con claves repetidas con mayor frecuencia) y otro con distribución uniforme. Mide en cada caso la tasa de falsos positivos de un Bloom filter estándar. Analiza cómo la distribución de la entrada afecta el rendimiento.

9. **Bloom filters jerárquicos**  
   - Diseña un esquema en el que varios Bloom filters de distinto tamaño y parámetros se encadenen (por ejemplo, un filtro "pequeño" para detección rápida y un filtro "grande" para mayor precisión). Explica cómo calcularías la tasa global de falsos positivos y evalúa la mejora en tiempo de consulta.

In [ ]:
## Tus respuestas

#### **Bloom filters como algoritmos aleatorizados**

Una vez que esas definiciones te queden claras, no debería ser difícil hacer una suposición educada: ¿A qué categoría pertenecen los Bloom filters?

Como probablemente ya habrás deducido, los Bloom filters son un ejemplo de estructura de datos Monte Carlo. El método `contains`, el algoritmo que comprueba si una clave está almacenada en un Bloom filter, en particular, es un algoritmo sesgado hacia los falsos positivos. De hecho, podría devolver verdadero para algunas claves que nunca se han añadido al filtro, pero siempre devuelve correctamente verdadero cuando una clave fue añadida previamente, por lo que no hay falsos negativos (es decir, cada vez que responde falso, estamos seguros de que la respuesta es correcta).

> Los Bloom filters también son un compromiso entre memoria y precisión. La versión determinista de un Bloom filter es un conjunto hash. 

### **Análisis de rendimiento**

Antes de comenzar con el análisis del Bloom filter, sugiero un estudio profundo de las métricas para algoritmos de clasificación que debes haber visto en cursos relacionados en inteligencia artificial.

Hemos visto cómo y por qué funcionan los Bloom filters; ahora veamos cuán eficientes son. Primero, examinaremos el tiempo de ejecución de las operaciones más importantes que ofrece un Bloom filter. A continuación,  abordaremos el desafío de predecir la precisión del método contains, dado un Bloom filter con una cierta estructura (en particular, su tamaño y el número de funciones hash utilizadas serán importantes).

#### **Tiempo de ejecución**

Ya hemos insinuado el hecho de que los Bloom filters pueden almacenar y buscar una clave en tiempo constante. Técnicamente, esto solo es cierto para entradas de longitud constante, aquí examinaremos el caso más genérico, cuando las claves almacenadas son cadenas de longitud arbitraria.

Pero comencemos desde el principio: la propia construcción de un Bloom filter. Después examinaremos en detalle insert y contains.

#### **Constructor**

La construcción de un Bloom filter es bastante sencilla; solo tenemos que inicializar un arreglo de bits, con todos sus elementos establecidos en 0, y generar el conjunto de `k` funciones hash. Hemos visto que la implementación también implica algún cálculo, pero es seguro marcar esa parte como de tiempo constante.

Crear e inicializar el arreglo requiere, obviamente, tiempo $O(m)$, mientras que generar cada una de las funciones hash típicamente requiere tiempo constante de ahí que, en total, se necesite $O(k)$ tiempo para generar todo el conjunto.

La construcción completa se puede terminar, en última instancia, en $O(m+k)$ pasos.

#### **Almacenando un elemento**

Para cada clave a almacenar, necesitamos producir `k` valores hash y cambiar un bit en cada uno de los elementos del arreglo indexado por esos resultados.

Haremos las siguientes suposiciones:

- Almacenar un solo bit requiere tiempo constante (posiblemente incluyendo el tiempo necesario para operaciones a nivel de bits en caso de usar búferes comprimidos para ahorrar espacio).
- Calcular el hash de una clave `X` requiere `T(X)` tiempo.
- El número de bits utilizados para almacenar una clave no depende del tamaño de la clave, ni del número de elementos ya añadidos al contenedor.

Dadas estas suposiciones, el tiempo de ejecución de `insert(X)` es `O(k*T(|X|))`. Para cada una de las `k` funciones hash que necesitamos, de hecho, se debe generar un valor hash a partir de la clave y almacenar un solo bit. Si las claves son números —por ejemplo, enteros o dobles, entonces `|X|=1` y `T(|X|) = O(1)`, lo que significa que típicamente podemos generar un valor hash en tiempo constante.

Si, sin embargo, nuestras claves tienen longitud variable —por ejemplo, cadenas— entonces calcular cada valor hash requiere tiempo lineal en la longitud de la cadena. En este caso, `T(|X|) = O(|X|)` y el tiempo de ejecución dependerá de la longitud de las claves que agreguemos.

Ahora, supongamos que sabemos que la clave más larga tendrá, como máximo, `z` caracteres, donde `z` es una constante. Recordando la suposición sobre que la longitud de las claves es independiente de cualquier otra cosa, podemos entonces argumentar que el tiempo de ejecución de `insert(X)` es `O(k*(1+z)) = O(k)`. Así, esta es una operación de tiempo constante, sin importar cuántos elementos se hayan añadido ya al contenedor.

#### **Buscando un elemento**

Las mismas consideraciones se aplican a la búsqueda de una clave: necesitamos transformar las claves en un conjunto de índices (lo cual se hace en tiempo `O(z*k)`), y luego verificar cada bit en esos índices (tiempo `O(k)` para todos ellos). Así, la búsqueda también es una operación de tiempo constante, bajo la suposición de que las longitudes de las claves están acotadas por una constante.

#### **Estimando la precisión del Bloom filter**

Antes de comenzar, necesitamos fijar algunas notaciones y hacer algunas suposiciones adicionales:

- `m` es el número de bits en nuestro arreglo.
- `k` es el número de funciones hash que usamos para mapear una clave a `k` posiciones diferentes en el arreglo.
- Cada una de las `k` funciones hash que usamos es independiente de las demás.
- El conjunto de funciones hash del cual extraemos nuestras `k` funciones es un conjunto de hashing universal.

Si se cumplen estas suposiciones, entonces se puede demostrar que una buena aproximación para la probabilidad de falsos positivos después de `n` inserciones viene dada por:

$$
\bigl(1 - e^{-kn/m}\bigr)^k
$$

donde $e$ es el número de Euler, la base de los logaritmos naturales.

Ahora tenemos una fórmula para estimar la probabilidad de un falso positivo. Eso es bueno por sí mismo, pero lo que es aún mejor es que nos permite ajustar `m` y `k`, los parámetros de nuestro Bloom filter. De esta manera, podemos decidir el tamaño del búfer del Bloom filter y el número de funciones hash necesarias para obtener una precisión óptima.

Hay tres variables en la fórmula para $p(n, m, k)$:

- `m`, el número de bits en el búfer  
- `n`, el número de elementos que se almacenarán en el contenedor  
- `k`, el número de funciones hash  

De esas tres, `k` es la que nos parece menos significativa. O, desde otro punto de vista, es la que está menos acoplada al problema. De hecho, `n` probablemente sea una variable que podamos estimar, pero no podemos controlar completamente. Puede que necesitemos almacenar tantos elementos como solicitudes recibamos, pero la mayoría de las veces podemos anticipar el volumen de solicitudes y hacer estimaciones conservadoras, para estar seguros.

También podríamos estar limitados en la elección de `m` ya que podríamos tener restricciones de memoria, tal vez no podamos usar más de `m` bits.

En cambio, para `k`, no tenemos ninguna restricción, y podemos ajustarlo para obtener una precisión óptima, es decir tenemos una probabilidad mínima de falsos positivos.

Afortunadamente, encontrar el valor óptimo para `k`, dado `m` y `n`, ni siquiera es tan difícil, es solo cuestión de encontrar el mínimo de la función:

$$
f(k) = -n \ln\bigl(1 - e^{-kn/m}\bigr)
$$

Nótese que `n` y `m` son constantes en la fórmula.

Entraremos en detalles luego sobre cómo encontrar el mínimo de $f(k)$, por ahora solo debes saber que el valor óptimo es:

$$
k^* = \frac{m}{n} \ln 2
$$

Ahora que tenemos una fórmula para $k^*$, podemos sustituirla en la fórmula anterior, la de $p(n, m, k)$, y tras algunas manipulaciones algebraicas, obtenemos una expresión para el valor óptimo de $m$ (llamémoslo $m^*$):

$$
m^* = -\frac{n \ln p}{(\ln 2)^2}
$$

Esto significa que si sabemos de antemano el número total $n$ de elementos únicos que insertaremos en el contenedor, y establecemos la probabilidad $p$ de un falso positivo al mayor valor aceptable, entonces podemos calcular el tamaño óptimo del búfer (y el número de bits por clave que necesitamos usar) para garantizar la precisión deseada.

Dos aspectos son significativos al observar las fórmulas que derivamos:

- El tamaño del búfer del Bloom filter es proporcional al número de elementos que se están insertando.  
- Al mismo tiempo, el número requerido de funciones hash solo depende de la probabilidad objetivo de falsos positivos $p$ (puedes ver esto sustituyendo $m^*$ en la fórmula para $k^*$).



#### **Ejercicios**

1. **Complejidad del constructor**  
   - Demuestra que inicializar el arreglo de bits y generar $k$ funciones hash lleva tiempo $O(m + k)$. ¿Qué suposiciones haces sobre la generación de cada función hash para considerarla de tiempo constante?

2. **Coste de `insert(X)`**  
   - Partiendo de que calcular el hash de una clave $X$ de longitud $|X|$ requiere tiempo $T(|X|)$, muestra que la complejidad de `insert(X)` es $O(k \cdot T(|X|))$. Luego, bajo la hipótesis de longitud de clave acotada por una constante $z$, explica por qué esto se convierte en $O(k)$.

3. **Coste de `contains(X)`**  
   - Razona por qué el coste de transformar una clave en índices es $O(z \cdot k)$ y la verificación de bits adyacentes es $O(k)$. Concluye la complejidad total de `contains(X)` bajo la suposición de $|X|\le z$.

4. **Derivación de la probabilidad de falsos positivos**  
   - A partir de las suposiciones de independencia y universalidad de los hashes, deriva paso a paso la fórmula  
     $$
       p(n,m,k)\approx\bigl(1 - e^{-kn/m}\bigr)^k.
     $$

5. **Optimización de parámetros**  
   - Muestra que minimizar  
     $\displaystyle f(k)=-n\ln\bigl(1-e^{-kn/m}\bigr)$  
     conduce a  
     $\displaystyle k^* = \frac{m}{n}\ln 2$.  
   - A continuación, usando $k^*$, deriva la expresión  
     $\displaystyle m^*=-\frac{n\ln p}{(\ln 2)^2}$.

6. **Sensibilidad a variaciones**  
   - Supón que tu estimación de $n$ se equivoca en un 20 %. Analiza cómo varían $k^*$ y $m^*$, y discute el impacto en la tasa de falsos positivos.

7. **Medición de tiempos de construcción**  
   - Con diferentes valores de $m$ (por ejemplo, $10^4,\,10^5,\,10^6$) y de $k$ (por ejemplo, $3,\,7,\,10$), mide empíricamente el tiempo de inicialización de un Bloom filter. Comprueba cómo crece el coste en función de $m+k$.

8. **Benchmark de inserción**  
   - Elige claves de longitud fija ($|X|=1$) y de longitud variable ($|X|\approx z$). Para varios $k$ (p. ej. $k=4,8$), mide el tiempo medio de `insert(X)` en cada caso y comprueba si coincide con la predicción $O(k)$ y $O(k\cdot|X|)$.

9. **Benchmark de búsqueda**  
   - Similar al anterior, mide el tiempo de `contains(X)` para claves de longitud acotada y distintos $k$. Grafica tiempo vs. $k$ y verifica la linealidad.

10. **Validación de la fórmula de falsos positivos**  
   - Para un filtro con parámetros conocidos $(m,n,k)$, inserta $n$ elementos y luego prueba $N$ claves nuevas no insertadas. Calcula la tasa empírica de falsos positivos y compárala con  
     $\bigl(1 - e^{-kn/m}\bigr)^k$.

11. **Reajuste dinámico de parámetros**  
   - Imagina un escenario en el que $n$ crece con el tiempo (p. ej. llegadas de flujo de datos). Diseña un protocolo para recalcular periódicamente $k$ (y, si fuera posible, reconstruir $m$) para mantener la tasa de falsos positivos por debajo de un umbral dado.

12. **Estudio de escalabilidad**  
   - Combina las mediciones anteriores para estimar cuántas inserciones y búsquedas por segundo puede soportar tu implementación antes de degradarse. Determina el **punto de inflexión** donde el coste de reconstrucción (por reajuste de parámetros) compensa el beneficio de una tasa de falsos positivos menor.


In [ ]:
## Tus respuestas

#### **Explicación de la fórmula de la tasa de falsos positivos**

En esta sección explicaremos con más detalle cómo se derivan las fórmulas para estimar la precisión de un Bloom filter, primero, veamos cómo obtenemos la estimación para la tasa de probabilidad de error.

Después de que se haya almacenado un solo bit en un Bloom filter con una capacidad de $m$ bits, la probabilidad de que un bit específico se establezca en 1 es $1/m$; entonces, la probabilidad de que el mismo bit se establezca en 0 después de que se hayan invertido los $k$ bits usados para almacenar un elemento (suponiendo que las funciones hash siempre generen $k$ valores diferentes para la misma entrada) es, por lo tanto,

$$
 \Bigl(1 - \tfrac{1}{m}\Bigr)^k
$$

Si consideramos los eventos de invertir cualquier bit específico a 1 como eventos independientes, entonces, después de insertar $n$ elementos, para cada bit individual en el búfer la probabilidad de que el bit aún sea 0 viene dada por

$$
p_{\text{bit}} = \Bigl(1 - \tfrac{1}{m}\Bigr)^{kn} \approx e^{-\frac{kn}{m}}
$$

Para tener un falso positivo, todos los $k$ bits correspondientes a un elemento $V$ deben haber sido establecidos en 1 de forma independiente, y la probabilidad de que todos esos $k$ bits sean 1 está dada por

$$
p(n, m, k) = (1 - p_{bit})^k = \left( 1 - \left( 1 - \frac{1}{m} \right)^{k \cdot n} \right)^k \approx \left( 1 - e^{-\frac{k \cdot n}{m}} \right)^k
$$


Lo cual es,  la fórmula de probabilidad mencionada anteriormente.

En este punto, podemos considerar $n$ y $m$ como constantes. Tiene sentido porque, en muchos casos, sabemos cuántos elementos necesitamos agregar al Bloom filter ($n$) y cuántos bits de almacenamiento podemos permitirnos ($m$). Lo que nos gustaría hacer es intercambiar rendimiento por precisión ajustando $k$, el número de funciones hash (universales) que usamos.

Esto equivale a encontrar el mínimo global de la función $f(k)$, definida como

$$
f(k) = \bigl(1 - e^{\frac{kn}{m}}\bigr)^k
$$

Si conoces algo de cálculo, probablemente ya hayas adivinado que necesitamos calcular las derivadas de $f$ con respecto a $k$.  Para facilitar el trabajo, podemos reescribir $f$ aplicando el logaritmo natural y la exponenciación, de modo que obtenemos


$$
f(k) = \left( 1 - e^{-\frac{k \cdot n}{m}} \right)^k = e^{k \cdot \ln\left(1 - e^{-\frac{k \cdot n}{m}}\right)}
$$

Esta función es mínima cuando su exponente es mínimo, por lo que podemos definir la función $g$

$$
g(k) = k \cdot \ln\left(1 - e^{-\frac{k \cdot n}{m}}\right)
$$

Y calculamos la derivada de $g$, la cual es facil para trabajar. La derivada de primer orden de $ g(k) $ es:

$$
g'(k) = \frac{\partial g}{\partial k} = \ln\left(1 - e^{-\frac{k \cdot n}{m}}\right) + k \cdot \frac{n}{m} \cdot \frac{-e^{-\frac{k \cdot n}{m}}}{1 - e^{-\frac{k \cdot n}{m}}}
$$

El mínimo ocurre cuando:

$$
k = \ln(2) \cdot \frac{m}{n}
$$

Para estar seguro del mínimo de la función, necesitamos calcular la segunda derivada y se verifica que la segundo derivada en ese punto es negativo:

$$
g''\left( \ln(2) \cdot \frac{m}{n} \right) < 0
$$


Cabe señalar que:

- La fórmula para $k$ nos da un valor único y exacto para la elección óptima del número de funciones hash.  
- $k$ obviamente debe ser un entero, por lo que el resultado necesita ser redondeado.  
- Valores mayores de $k$ implican peor rendimiento para `insert` y `lookup` (porque se deben calcular más funciones hash para cada elemento), por lo que se puede preferir un compromiso con valores de $k$ ligeramente menores.  

Si usamos el mejor valor de $k$ como se calculó previamente, esto significa que la tasa de falsos positivos $f$ se convierte en:

$$
f = \left( \frac{1}{2} \right)^k = (0.6185)^{\frac{m}{n}}
$$

Al reemplazar el valor de $k$ en la fórmula de $p(n, m, k)$, podemos obtener una nueva fórmula que relaciona el número de bits de almacenamiento con el número (máximo) de elementos que se pueden almacenar, independientemente de $k$ (el valor de $k$ puede calcularse después) para garantizar una probabilidad de falso positivo menor que un cierto valor $p$:

$$
p = p(n, m) = \left( 1 - e^{-\left( \frac{m}{n} \cdot \ln(2) \right)} \right)^{\frac{m}{n} \cdot \ln(2)} = \left( \frac{1}{2} \right)^{\frac{m}{n} \cdot \ln(2)}
$$

Tomando el logaritmo en base 2 en ambos lados:

$$
\log_2(p) = \log_2\left( \left( \frac{1}{2} \right)^{\frac{m}{n} \cdot \ln(2)} \right) = -\frac{m}{n} \cdot \ln(2)
$$

y luego resolviendo para $m$ finalmente obtenemos:

$$
m^* = -n \cdot \frac{\log_2(p)}{\ln(2)} = -n \cdot \frac{\ln(p)}{(\ln(2))^2}
$$

Esto significa que, si conocemos de antemano el número total $n$ de elementos únicos que insertaremos en el Bloom filter, y fijamos la probabilidad de falso positivo $p$ al valor máximo aceptable, entonces podemos calcular el tamaño óptimo del buffer que garantiza la precisión deseada. También tendremos que establecer $k$ en consecuencia, utilizando la fórmula que derivamos anteriormente.


$$
k^* = \ln(2) \cdot \frac{m^*}{n} = -\ln(2) \cdot n \cdot \frac{\ln(p)}{(\ln(2))^2} \cdot \frac{1}{n} = \frac{\ln(p)}{\ln(2)}
$$

Por ejemplo, si quisiéramos tener una precisión del 90%, y por lo tanto a lo sumo un 10% de falsos positivos, podemos sustituir los valores y obtener:

$$
k = \frac{-\ln(0.1)}{\ln(2)} = \frac{-2.3025}{0.6931} = 3.32
$$

$$
m = -n \cdot \frac{\ln(p)}{(\ln(2))^2} = 4.792 \cdot n
$$

